# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
filepath = os.getcwd() + '/event_data'

for root, dirs, files in os.walk(filepath):
    file_path_list = glob.glob(os.path.join(root,'*'))

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
full_data_rows_list = [] 
    
for f in file_path_list:

    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
           
        for line in csvreader:
            full_data_rows_list.append(line) 
            
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

In [4]:
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
from cassandra.cluster import Cluster

try:
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [6]:
try:
    session.execute("""
        CREATE KEYSPACE IF NOT EXISTS sparkify
        WITH REPLICATION = 
        {'class': 'SimpleStrategy',  'replication_factor': 1}
    """)
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




#### Query 1

#### We will create a session_table with following columns: session_id, item_in_session, artist_name, song_title and song_lenght. Since session_id and item_in_session will be used in where clause, these two will be our partition key.

In [8]:
query = "CREATE TABLE IF NOT EXISTS session_table "
query = query + "(session_id int, item_in_session int, artist_name text, song_title text, song_length float, PRIMARY KEY (session_id, item_in_session))"

try:
    session.execute(query)
except Exception as e:
    print(e)


In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:
        query = "INSERT INTO session_table (session_id, item_in_session, artist_name, song_title, song_length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        try:
            session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))
        except Exception as e:
            print(e)

#### We will use SELECT to verify that the data have been inserted into each table

In [10]:
query = """
        SELECT artist_name, song_title, song_length FROM session_table WHERE session_id = 338 and item_in_session = 4
        """
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print((row.artist_name, row.song_title, row.song_length))

('Faithless', 'Music Matters (Mark Knight Dub)', 495.30731201171875)


#### Query 2
#### We will create a user_session_table. Since our query searches for userId and sessionId, these two will be our partition key. We will include itemInSession as clustering key for sorting of this field

In [11]:
query = "CREATE TABLE IF NOT EXISTS user_session_table "
query = query + """
                (user_id int, session_id int, item_in_session int, artist_name text, song_title text, first_name text, last_name text,
                PRIMARY KEY ((user_id, session_id), item_in_session))
                """
try:
    session.execute(query)
except Exception as e:
    print(e)           

In [12]:
with open(file, encoding='utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    
    for line in csvreader:
        query = """
                INSERT INTO user_session_table (user_id, session_id, item_in_session, artist_name, song_title, first_name, last_name)
                """
        
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
                    
        try:
            session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))
        except Exception as e:
            print(e)

#### We will use SELECT to verify that the data have been inserted into each table

In [13]:
query = """
        SELECT artist_name, song_title, first_name, last_name 
           FROM user_session_table
           WHERE user_id = 10 and session_id = 182
        """
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print((row.artist_name, row.song_title, row.first_name, row.last_name))

('Down To The Bone', "Keep On Keepin' On", 'Sylvie', 'Cruz')
('Three Drives', 'Greece 2000', 'Sylvie', 'Cruz')
('Sebastien Tellier', 'Kilometer', 'Sylvie', 'Cruz')
('Lonnie Gordon', 'Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', 'Sylvie', 'Cruz')


#### Query 3
#### We will create a user_table. Since our query searches for song, first_name and last_name of the user, we will use song_title and user_id as partition key to ensure that per row is unique.

In [14]:
query = "CREATE TABLE IF NOT EXISTS user_table "
query = query + "(song_title text, user_id text, first_name text, last_name text, PRIMARY KEY(song_title, user_id))"

try:
    session.execute(query)
except Exception as e:
    print(e)

In [15]:
with open(file, encoding='utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    
    for line in csvreader:
        query = """
                INSERT INTO user_table (song_title, user_id, first_name, last_name)
                """
        query = query + "VALUES (%s, %s, %s, %s)"
                    
        try:
            session.execute(query, (line[9], line[10], line[1], line[4]))
        except Exception as e:
            print(e)

#### We will use SELECT to verify that the data have been inserted into each table

In [16]:
query = """
        SELECT first_name, last_name 
           FROM user_table
           WHERE song_title = 'All Hands Against His Own' 
        """
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print((row.first_name, row.last_name))

('Jacqueline', 'Lynch')
('Tegan', 'Levine')
('Sara', 'Johnson')


### Drop the tables before closing out the sessions

In [17]:
queries = ["DROP TABLE IF EXISTS session_table",
           "DROP TABLE IF EXISTS user_session_table",
           "DROP TABLE IF EXISTS user_table"]

for query in queries:
    try:
        session.execute(query)
    except Exception as e:
        print(e)

### Close the session and cluster connection¶

In [18]:
session.shutdown()
cluster.shutdown()